### 3. KOBIS OPENAPI

영화관 입장권 통합 전산망 OPENAPI를 사용하여 일일 박스오피스 자료를 가져오고 JSON파일로 저장

--------------------------------------------------
### API 활용
--------------------------------------------------

In [1]:
import os
import requests
import json
import calendar
from tqdm import tqdm
from tqdm import tqdm_notebook
from datetime import date, timedelta

key='ad816991534afeaef71f07e7336b0d61'
movie_info_path='/root/movie/data/'

In [2]:
def check_dir(start_date,end_date):    
    start_year = int(start_date.year)
    end_year = int(end_date.year)
    year = start_year
    for i in tqdm(range(start_year,end_year+1)):        
        if(os.path.isdir(movie_info_path+str(year))!=1):
            os.mkdir(movie_info_path+str(year)+'/')
            print("create "+str(year)+" folder") 
        year += 1

연도별 폴더가 존재하는지 확인한 뒤 없으면 생성

In [3]:
def daily_box_office_to_json(start_date,end_date):    
    tmp_date = start_date
    startDt = int(str(start_date)[0:4]+str(start_date)[5:7]+str(start_date)[8:10])
    endDt = int(str(end_date)[0:4]+str(end_date)[5:7]+str(end_date)[8:10])
    days = end_date - start_date
    for i in tqdm(range(0,days.days+1)):
        targetDt = int(str(tmp_date)[0:4]+str(tmp_date)[5:7]+str(tmp_date)[8:10])
        
        if(os.path.isfile("/root/movie/data/"+str(targetDt)[0:4]+'/'+str(targetDt)+'.json')!=1):
            url='http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key='+key+'&targetDt='+str(targetDt)
            res = requests.get(url)
            text = res.text
            d = json.loads(text)
            with open('data/'+str(targetDt)[0:4]+'/'+str(targetDt)+'.json', 'w', encoding="utf-8") as make_file:
                json.dump(d, make_file, ensure_ascii=False, indent="\t")
        tmp_date += timedelta(1)

일일 박스오피스 자료를 찾고 없으면 API를 통해 자료를 받아온다. 파일은 해당일을 이름으로 지정하고 JSON 타입으로 폴더에 저장

In [4]:
start_date = date(2017,1,1)
end_date = date(2017,12,31)
check_dir(start_date,end_date)
daily_box_office_to_json(start_date,end_date)

100%|██████████| 365/365 [00:00<00:00, 58689.71it/s]


샘플 데이터로 2017년 전체 자료를 받아온다

In [5]:
import pandas as pd
import requests
import datetime
import json
from tqdm import tqdm_notebook
from dateutil.relativedelta import relativedelta

my_date = datetime.date(2017, 1, 1)

In [6]:
start_date = date(2017,1,1)
targetDt = str(start_date)[0:4]+str(start_date)[5:7]+str(start_date)[8:10]
with open('data/'+str(targetDt)[0:4]+'/'+str(targetDt)+'.json', 'r', encoding="utf-8") as read_file:
    d = json.load(read_file)

In [7]:
def daily_boxoffice(start_date, end_date) :
    
    final_list = []

    for single_date in tqdm_notebook(pd.date_range(start_date, end_date)) :
        targetDt = str(single_date)[0:4]+str(single_date)[5:7]+str(single_date)[8:10]
        with open('data/'+str(targetDt)[0:4]+'/'+str(targetDt)+'.json', 'r', encoding="utf-8") as read_file:
            d = json.load(read_file)

        for item in d["boxOfficeResult"]["dailyBoxOfficeList"] :
            value_list = []
            key_list = []
            for key, value in item.items() :
                key_list.append(key)
                value_list.append(value)

            value_list.append(single_date)
            key_list.append('CurrentDate')
            final_list.append(value_list)

    return pd.DataFrame(final_list, columns=key_list)

In [8]:
start_date = date(2017,1,1)
end_date = date(2017,12,31)
a = daily_boxoffice(start_date, end_date)

### 일별 박스 오피스를 월별로 추출, 영화코드 리스트 추출


In [9]:
movie_code_list_all=pd.DataFrame()
for i in tqdm_notebook(range(12)):
    start_date = my_date + relativedelta(months=i)
    end_date = start_date+ relativedelta(months=1) - datetime.timedelta(days=1)
    
    daily_boxoffice_data = daily_boxoffice(start_date, end_date)
    startdate=start_date.strftime('%Y%m%d')
    
    enddate=end_date.strftime('%Y%m%d')
    
    movie_code_list = pd.DataFrame({'movieCd' :daily_boxoffice_data.movieCd.drop_duplicates(), 'movieNm':daily_boxoffice_data.movieNm.drop_duplicates(), 'director':'','Rate' : 0})
    
    
    daily_boxoffice_data.to_csv("daliy_boxoffice_data_"+startdate+"~"+enddate+".csv", index=False)
    movie_code_list.to_csv("movie_code_list_"+startdate+"~"+enddate+".csv", index=False)
    
    movie_code_list_all = movie_code_list.append(movie_code_list_all)
    movie_code_list_all = movie_code_list_all.drop_duplicates()
    movie_code_list_all.to_csv("movie_code_list_all.csv", index=False)

--------------------------------------------------------------
영화 코드를 통해 영화 정보 파일 불러오기
--------------------------------------------------------------

In [10]:
def movie_info_api(df):
    for i in range(0,len(df['movieCd'])):
        movieCd = str(df['movieCd'][i])

        if(os.path.isfile("/root/movie/movie_info/"+movieCd+".json")):
#             print(movieCd+".json already exist!")
            pass
        else:
            url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key='\
                +key+'&movieCd='+movieCd    
            res = requests.get(url)
            text = res.text
            d = json.loads(text)
#             print(movieCd+".json is created!")
            with open('movie_info/'+movieCd+'.json', 'w', encoding="utf-8") as make_file:
                json.dump(d, make_file, ensure_ascii=False, indent="\t")

In [11]:
df = pd.read_csv('movie_code_list_all.csv')
movie_info_api(df)

In [12]:
#영화데이터  : txt 로 저장 함수 정의
def movie_data(lists) :    
    dict_list = dict()

    for movie in lists :
        with open('movie_info/'+movie+'.json', 'r', encoding="utf-8") as read_file:
            d = json.load(read_file)
        
        dict_list[movie] = d['movieInfoResult']['movieInfo'] #dict_list[movie] 가 있어야 append형식으로 됨. 없으면 마지막것만 추출
    return dict_list

In [13]:
movie_info=movie_data(movie_code_list_all.movieCd) 
with open('movie_info.txt', 'w', encoding='utf-8') as outfile:   #딕셔너리를 json으로 저장
    json.dump(movie_info, outfile, ensure_ascii=False)

In [15]:
movie_code_list_all = pd.read_csv("movie_code_list_all.csv")
for i in range(len(movie_code_list_all)):
    with open('movie_info/'+str(movie_code_list_all['movieCd'][i])+'.json', 'r', encoding="utf-8") as make_file:
        d = json.load(make_file)
    if(d['movieInfoResult']['movieInfo']['directors'] != []):
        movie_code_list_all['director'][i]=d['movieInfoResult']['movieInfo']['directors'][0]['peopleNm']
        
    else:
        movie_code_list_all['director'][i] = '---'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [16]:
movie_code_list_all.to_csv("movie_code_add_director.csv")